# Pre-Processing Pipeline

In [1]:
#Given pseudocode to grab the .h5 data file
import numpy as np
import h5py
from scipy import signal
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

with h5py.File("/home/ljian/private/PHYS 139 Final Project/deepclean-1251335314-4097.h5", "r") as f:
    X = []
    for channel, timeseries in f.items():
        if channel == "H1:GDS-CALIB_STRAIN":
            y = timeseries[:]
        else:
            X.append(timeseries[:])
X = np.stack(X, axis=-1) # this will do channels last (as is typically done for Keras)
print(X.shape)
print(y.shape)

(16781312, 21)
(16781312,)


## Test Train Split

In [ ]:
train_max_index = int(X.shape[0] * 4 / 5)

X_train = X[:train_max_index]
y_train = y[:train_max_index]

X_test = X[train_max_index:]
y_test = y[train_max_index:]

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

## Batching (Train and Test)

In [ ]:
# then to construct inputs/outputs, you just
# slice windows from these two timeseries

# window_size = 8192 to replicate data size for paper
# Can change this around to try different shapes
window_length = 2 # s
sample_rate = 4096 # Hz
window_size = window_length * sample_rate

# train_batches in your code is currently 1000, which is fine, 
# but i set it here to int(total time steps / window size).


# Train data batching
train_batches = int(X_train.shape[0]/window_size) 
X_train_batch, y_train_batch = [], []
for i in range(train_batches):
    idx = np.random.randint(X_train.shape[0] - window_size)
    X_train_batch.append(X_train[idx: idx + window_size])
    y_train_batch.append(y_train[idx: idx + window_size])

X_train_batch = np.stack(X_train_batch)
y_train_batch = np.stack(y_train_batch)

print(X_train_batch.shape)
print(y_train_batch.shape)



# Test data batching
# the idea here is the test data should be fixed and not randomized/shuffled
# so we're just going to grab as many contiguous slices of it as we can
test_batches = int(X_test.shape[0]/window_size)
X_test_batch, y_test_batch = [], []
for i in range(test_batches):
    idx = i * window_size
    X_test_batch.append(X_test[idx: idx + window_size])
    y_test_batch.append(y_test[idx: idx + window_size])

X_test_batch = np.stack(X_test_batch)
y_test_batch = np.stack(y_test_batch)

print(X_test_batch.shape)
print(y_test_batch.shape)


plt.plot(y_train_batch[0:100])
plt.title("Strain Data")
plt.show()

plt.plot(X_train_batch[0][:,0])
plt.title("First Aux Channel. First Batch")
plt.show()


# Train Data
strain_data_train = y_train_batch
witness_data_train = X_train_batch

# Test Dats
strain_data_test = y_test_batch
witness_data_test = X_test_batch

channels = 21

#may be the wrong size
#Ask author what size to train model on, sample more finely doesnt have to be most 32k, try (8192, 21) slices; would
#likely be 2s of data.
#possibly increase batch size
#have to train_test_split earlier, before bringing into cnn_model

## Pre-processing data structure:

1. 8th Order Butterworth: Aliasing filter to mitigate power from outside the witnessed noise frequencies;

2. StandardScalar: Unit variance, Zero mean;

3. Windowing: Divide data into smaller overlapping windows;
 * Segment Length: 8.00 Seconds;
 * 0.5 Hz => DFT of 2 Seconds;
    
4. Welches Method: 1 Second DFT overlap;

5. Training: Segment overlap [%] = 96.875

6. Testing: Segment overlap [%] = 50%

## PSD and CSD analysis

In [ ]:
# 2.0: Butterworth
print("Finding pass-band frequencies via Cross Spectral Density Analysis")
# Find band pass values per channel

for i in range(0,21):
    plt.csd(X[:,i], y, Fs = 4096)
    plt.title("CSD over Channel i and Strain")

plt.xscale("log")
plt.show()


print("Finding pass-band frequencies via Cross Spectral Density Analysis")
## Find band pass values per channel
for i in range(0,21):
    plt.psd(X[:,i], Fs = 4096)
    plt.title("PSD over all Channels")

plt.xscale("log")
plt.show()

In [ ]:
print("Building Butterworth Filter")
N = 8 # order of butterworth filter

Wn = [0, 0.3] # Band-pass [low end, high end]
# Wn = [10, 100] # Band-pass [low end, high end]

filter_type = "bp"
fs = Wn[1]/2

b, a = signal.butter(N, Wn, filter_type, analog=True)
w, h = signal.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()
butter_sos = signal.butter(N, Wn, filter_type, fs , output = 'sos')
print("Butterworth filter built")

In [ ]:
# 2.1: Apply butter_sos to data
print("Applying Butterworth to data")
print("Train Witness shape:", witness_data_train.shape)
print("test Witness shape:", witness_data_test.shape)

# Pre-filtered plot
plt.plot(witness_data[0][:,0])
plt.show()

# Filter Train Data
for idx in range(len(witness_data_train)):
    for channel in range(channels):
        witness_data_train[idx][:, channel] = signal.sosfilt(butter_sos, witness_data_train[idx][:, channel])

# Filter Test Data
for idx in range(len(witness_data_test)):
    for channel in range(channels):
        witness_data_test[idx][:, channel] = signal.sosfilt(butter_sos, witness_data_test[idx][:, channel])        
        
# Post-filtered plot        
plt.plot(witness_data_train[0][:,0])
plt.show()
print("Butterworth filter applied to data")

## 3: Standard Scalar (Z-score)

In [ ]:


print("Normalization in progress")

# Train Data
plt.plot(witness_data_train[0][:,0])
plt.show()

scaler = StandardScaler()
strain_data_train = np.reshape(strain_data_train, (-1,1))

## For Post Processing
strain_mean_train = np.mean(strain_data_train)
strain_std_train = np.std(strain_data_train)


scaler.fit(strain_data_train)
strain_data_train = scaler.transform(strain_data_train)


test_1 = scaler.transform(np.reshape(witness_data_train[1][:,0], (-1,1)))
plt.plot(test_1, label = "Z_score")
plt.plot(witness_data_train[0][:,0], label = "OG Data")
plt.legend()
plt.show()




# Test Data
plt.plot(witness_data_test[0][:,0])
plt.show()

scaler = StandardScaler()
strain_data_test = np.reshape(strain_data_test, (-1,1))

## For Post Processing
strain_mean_test = np.mean(strain_data_test)
strain_std_test = np.std(strain_data_test)


scaler.fit(strain_data_test)
strain_data_test = scaler.transform(strain_data_test)


test_1 = scaler.transform(np.reshape(witness_data_test[1][:,0], (-1,1)))
plt.plot(test_1, label = "Z_score")
plt.plot(witness_data_test[0][:,0], label = "OG Data")
plt.legend()
plt.show()

print("CNN Data Ready, Import witness_data")
print(witness_data_train.shape)



print("Normalization Complete")

## 4: Windowing Parameters

In [ ]:
# 4: Windowing
# Should match window_length from Batching cell
window_length = 2
DFT_length = 2
sample_rate = 4096
window_size = window_length * sample_rate
DFT_size = DFT_length * sample_rate
overlap_percent = 0.96875



## 5: Welch's Method and DFT

In [ ]:
# 5: Welches Method
print("Applying Welch's Method")
window_type = 'hann'
nperseg_train = window_size
noverlap_train = window_size*(1-overlap_percent)
nfft = DFT_size
freq_data_train = []
freq_data_test = []

# Train Data
for batch in range(len(witness_data_train)):
    batch_data = []
    for channel in range(channels):
        welch_data = signal.welch(witness_data_train[batch][:, channel], sample_rate, window = 'hann')
        batch_data.append(welch_data[1])
    batch_data = np.stack(batch_data, axis = -1)
    freq_data_train.append(batch_data)
freq_data_train = np.stack(freq_data, axis = 0)

# Test data
for batch in range(len(witness_data_test)):
    batch_data = []
    for channel in range(channels):
        welch_data = signal.welch(witness_data_test[batch][:, channel], sample_rate, window = 'hann')
        batch_data.append(welch_data[1])
    batch_data = np.stack(batch_data, axis = -1)
    freq_data_test.append(batch_data)
freq_data_test = np.stack(freq_data, axis = 0)

print("Welch's Method Complete")


## API Call to send data

In [ ]:
def send_freq_data():
    """
    new_data == Spectral density data data,
    strain_data == GW processed data
    
    Used for loss function computaiton
    """
    return freq_data_train, strain_data_train, freq_data_test, strain_data_test

def send_time_data():
    """
    Train Data: 
    witness_data_train == filtered, and z_scored aux channel data,
    strain_data_train == GW processed data
    
    Test Data: 
    witness_data_test == filtered, and z_scored aux channel data,
    strain_data_test == GW processed data
    
    Used for CNN training. Not split yet.
    """
    return witness_data_train, strain_data_train, witness_data_test, strain_data_test


def send_post_processing_params():
    """
    Train Data
    strain_mean_train == mean of data used for z-score
    strain_std_train = std dev used for z-score
    
    Test Data
    strain_mean_test == mean of data used for z-score
    strain_std_test = std dev used for z-score
    
    Will be needed in post processing. Call function to return the old mean and std dev to reverse z-score
    """
    return strain_mean_train, strain_std_train, strain_mean_test, strain_std_test